# VacationPy
----


In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:

# Create a reference the CSV file desired
csv_path = "output_data/cities.csv"

# Read the CSV into a Pandas DataFrame
cities_df= pd.read_csv(csv_path)

# Print the first five rows of data to the screen
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hilo,19.7297,-155.0900,76.73,86,90,10.36,US,1627103690
1,High Level,58.5169,-117.1360,66.31,72,90,3.44,CA,1627103706
2,Port Alfred,-33.5906,26.8910,49.51,67,36,8.08,ZA,1627103706
3,Tuatapere,-46.1333,167.6833,51.82,82,88,4.21,NZ,1627103706
4,Hong Kong,22.2855,114.1577,96.96,78,5,1.99,HK,1627103655


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"]

In [28]:
#Add Heatmap layer to map.
# Plot Heatmap
fig = gmaps.figure(center=(22.0, -1.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights = weight, 
                                 dissipating=False, max_intensity=np.max(weight),
                                 point_radius=3)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
#   * A max temperature lower than 80 degrees but higher than 70.

#   * Wind speed less than 10 mph.

#   * Zero cloudiness.

#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Dropped cities with missing values
cities_new_df = cities_df.dropna()
cities_new_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hilo,19.7297,-155.0900,76.73,86,90,10.36,US,1627103690
1,High Level,58.5169,-117.1360,66.31,72,90,3.44,CA,1627103706
2,Port Alfred,-33.5906,26.8910,49.51,67,36,8.08,ZA,1627103706
3,Tuatapere,-46.1333,167.6833,51.82,82,88,4.21,NZ,1627103706
4,Hong Kong,22.2855,114.1577,96.96,78,5,1.99,HK,1627103655


In [18]:
#   * A max temperature lower than 80 degrees but higher than 70.

#   * Wind speed less than 10 mph.

#   * Zero cloudiness.
filtered_cities = cities_new_df.loc[(cities_new_df["Max Temp"] <= 80) & 
                                    (cities_new_df["Max Temp"] >= 70) & 
                                    (cities_new_df["Wind Speed"] < 10) &  
                                    (cities_new_df["Cloudiness"] == 0)]
                                
filtered_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,Vao,-22.6667,167.4833,71.20,80,0,8.16,NC,1627103721
94,Cayenne,4.9333,-52.3333,79.09,86,0,2.30,GF,1627103592
210,Abhar,36.1468,49.2180,72.25,55,0,6.67,IR,1627103816
257,Tulun,54.5611,100.5792,76.39,45,0,9.95,RU,1627103846
292,Ajdabiya,30.7554,20.2263,74.62,71,0,6.96,LY,1627103862
425,Poum,-20.2333,164.0167,74.80,82,0,7.02,NC,1627103947
469,Natal,-5.7950,-35.2094,73.62,83,0,8.05,BR,1627103840
518,Awjilah,29.1081,21.2869,76.89,47,0,9.10,LY,1627104004
539,Balagansk,54.0103,103.0544,72.48,47,0,4.56,RU,1627104017


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df = pd.DataFrame(filtered_cities)
hotel_df["Hotel Name"] = ""

In [20]:
# geocoordinates
target_search = "hotels"
#Set parameters to search for hotels with 5000 meters.
target_radius = 5000
target_type = "Business"

# params dictionary to update each iteration
params = {
    "keyword": target_search,
    "radius":target_radius,
    "type":target_type,
    "key" : g_key
}
    
# Use the lat/lng we recovered to identify 
for index, row in hotel_df.iterrows():
# Use the search term: "hotels" and our lat/ln
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # make request and print url
    hotel_info = requests.get(base_url, params=params)
    hotel_info = hotel_info.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_info["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = hotel_info["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = hotel_info["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
       

Missing field/result... skipping.
Missing field/result... skipping.


In [21]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address,Hotel Rating
29,Vao,-22.6667,167.4833,71.20,80,0,8.16,NC,1627103721,OURE TERA beach resort,Kuto,4.3
94,Cayenne,4.9333,-52.3333,79.09,86,0,2.30,GF,1627103592,Mercure Cayenne Royal Amazonia,"45 Rue de l'Ara Bleu, Cayenne",4.1
210,Abhar,36.1468,49.2180,72.25,55,0,6.67,IR,1627103816,Hotel Got7,استان زنجان، ابهر، 18 متری پردیس،,0.0
257,Tulun,54.5611,100.5792,76.39,45,0,9.95,RU,1627103846,Gostinitsa Tsentral'naya,"Ulitsa Lenina, 122, Tulun",3.7
292,Ajdabiya,30.7554,20.2263,74.62,71,0,6.96,LY,1627103862,فندق التوجان,شارع انتلات، Ajdabiya,3.4
425,Poum,-20.2333,164.0167,74.80,82,0,7.02,NC,1627103947,,NaN,NaN
469,Natal,-5.7950,-35.2094,73.62,83,0,8.05,BR,1627103840,Villa Park Hotel | Natal,"Av. Sen. Salgado Filho, 1525 - Tirol, Natal",4.4
518,Awjilah,29.1081,21.2869,76.89,47,0,9.10,LY,1627104004,,NaN,NaN
539,Balagansk,54.0103,103.0544,72.48,47,0,4.56,RU,1627104017,Sberbank,"Kol'tsevaya Ulitsa, 61, Balagansk",2.0


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content= hotel_info)
# Display figure
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))

In [24]:
#Plotting the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))